# pytorch LSTM

github上可以查到各种版本的LSTM，在使用它们之前，有必要了解下pytorch本身自带的lstm，本文就以pytorch自身的LSTM为例，另外补充一些基于它的一些repo。这部分主要参考了：[SEQUENCE MODELS AND LONG-SHORT TERM MEMORY NETWORKS](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html?highlight=lstm)。

序列建模中，不同时间之间的输入是有依赖关系的。经典的序列建模的例子就是隐马尔可夫模型，还有条件随机场。循环神经网络是一种能保存一些状态state的神经网络。它的输出能被用作下一个输入的一部分，因此信息能够随着网络沿序列传播下去。在LSTM中，序列中的每个元素都有对应的隐含状态，它能序列中它之前的点的信息。我们可以用隐含状态来帮助预测。

pytorch的LSTM需要所有的输入是三维张量。张量的每个轴的含义是十分重要的，不可搞错。pytorch lstm是sequence first的。即第一维是序列本身，第二维是一个batch中的东西，第三是输入向量的维度。再具体点说，是这样的：

pytorch 的LSTM单元接收的输入必须是三维张量：

- 第一维反映序列seq的结构，即seq内的个数
- 第二维是minibatch，即一次喂给网络的seq数目
- 第三维是输入元素，每个元素具体是多少维的向量

第一维和第二维和直观上的维度顺序不太一致，因为 pytorch 对tensor是sequence first排列维度的。

- batch first: $a_1 a_2 a_3 a_4 a_5|b_1 b_2 b_3 b_4 b_5$
- sequence first: $a_1 b_1| a_2 b_2| a_3 b_3| a_4 b_4| a_5 b_5$

所以pytorch中不同seq同一时刻对应的输入单元在内存中毗邻，这样可以快速读取数据。

关于lstm的基本概念可以参考：https://github.com/OuyangWenyu/elks/blob/master/math-basics/nn/rnn/lstm.ipynb

现在看一个例子。

In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn # 神经网络模块
import torch.nn.functional as F # 神经网络模块中的常用功能
import torch.optim as optim   # 模型优化器模块

torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[ 0.2490, -0.0525,  0.3253]],

        [[ 0.1655, -0.0304,  0.3348]],

        [[-0.1104, -0.1085,  0.7568]],

        [[-0.0148, -0.0855,  0.4162]],

        [[ 0.0703, -0.1089,  0.2071]]], grad_fn=<StackBackward>)
(tensor([[[ 0.0703, -0.1089,  0.2071]]], grad_fn=<StackBackward>), tensor([[[ 0.2099, -0.3541,  0.9947]]], grad_fn=<StackBackward>))


Pytorch中，LSTM类继承自RNNBase类，RNNBase类继承自Module类，Module类是pytorch中完成一定网络功能的基类，可以通过继承该类定义自己的神经网络。自己实现神经网络时，一般要重写其forward方法。

Module实现了__call__方法，这意味着其可被当做可调用方法使用。比如上面就有直接用到lstm()。

接下来简单分析下上面的代码。

首先，固定随机种子，这样后面的结果能维持不变，这也是实现reproducible结果的必要手段。

然后，定义LSTM网络，定义的时候使用的是输入和输出的维度，这里这个维度是LSTM网络输入输出的三维张量的最后一维的大小，也就是向量本身的长度。

再然后，生成一个长度为5，每一个元素为1\*3的序列作为输入，即输入张量的维度为(5,1,3)。

接下来设置隐藏层维度，初始化隐藏层的数据。hidden变量是一个**元组**，其第一个元素是LSTM**隐藏层输出**，另一个元素维护**隐藏层的状态**。每个元素都是torch.rand(1,1,3)，就是一个维度为(1,1,3)的以一定高斯分布生成的张量。三维的含义和输入的是一致的。

接下来的前向计算有两种方式。

一是沿序列一次计算一步，每步计算之后，hidden变量都会包含hidden state。

另一种是一次计算整个序列，cat函数可以参考：https://pytorch.org/docs/stable/torch.html?highlight=cat#torch.cat ，简而言之就是拼接功能，将输入拼接起来，并转为一个三维(len(inputs), 1, -1)的张量。